In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import chardet
import os
import json
import nltk
import re
import glob
from io import StringIO
from nltk.tokenize import sent_tokenize, word_tokenize

# Download the necessary Punkt Tokenizer Models
nltk.download('punkt', quiet=True)
try:
    nltk.download('punkt_tab', quiet=True)  # Tentiamo di scaricare anche punkt_tab
except:
    print("Impossibile scaricare punkt_tab, utilizzo punkt standard")

# Assicuriamoci che il tokenizer sia disponibile
if not nltk.data.find('tokenizers/punkt'):
    raise RuntimeError("Impossibile trovare il tokenizer punkt di NLTK. Verificare l'installazione di NLTK.")

# Impostazioni per la segmentazione
MAX_LEN = 384  # Lunghezza massima di un segmento
OVERLAP = 100  # Sovrapposizione tra segmenti
PUNCTUATION = [".", "!", "?", ";", ":", ","]  # Punti di spezzatura preferiti
CHUNK_SIZE = 1024 * 1024  # Dimensione del blocco di lettura (1 MB)

# Percorsi dei file di output
output_txt_path = "/content/unified_bandi.txt"
output_json_path = "/content/unified_bandi.json"
output_segments_path = "/content/testo_segmentato_nltk.txt"

# Lista di file caricati - gestione delle sottocartelle
document_files = []
# Ottieni tutte le sottocartelle all'interno di /content/plain/
subdirs = glob.glob("/content/plain/*/")
for subdir in subdirs:
    # Per ogni sottocartella, ottieni tutti i file plain_*
    files_in_subdir = glob.glob(os.path.join(subdir, "plain_*"))
    document_files.extend(files_in_subdir)

# Se non ci sono sottocartelle o per sicurezza, verifica anche i file direttamente in /content/plain/
direct_files = glob.glob("/content/plain/plain_*")
document_files.extend(direct_files)

# Rimuovi eventuali duplicati
document_files = list(set(document_files))

def detect_encoding(file_path):
    """Rileva l'encoding corretto del file."""
    with open(file_path, 'rb') as f:
        raw_data = f.read(CHUNK_SIZE)
        result = chardet.detect(raw_data)
        return result['encoding']

def clean_text(text):
    """Effettua la pulizia del testo rimuovendo caratteri speciali e spazi extra."""
    text = re.sub(r'[^\w\s.,!?]', '', text)  # Rimuove caratteri non alfanumerici tranne punteggiatura
    text = re.sub(r'\s+', ' ', text).strip()  # Rimuove spazi extra
    return text

def find_best_split(words, max_len):
    """Trova il miglior punto di spezzatura in base alla punteggiatura."""
    if len(words) <= max_len:
        return len(words)  # Nessuna spezzatura necessaria
    best_split = max_len  # Punto di spezzatura predefinito
    for i in range(max_len - 10, max_len - 50, -1):  # Cerca un punto vicino alla fine
        if i < 0 or i >= len(words):  # Controllo aggiuntivo per sicurezza
            continue
        if words[i] in PUNCTUATION:
            best_split = i + 1  # Include la punteggiatura nel segmento
            break
    return best_split

def segment_text(words):
    """Segmenta il testo in blocchi di massimo MAX_LEN parole."""
    segments = []
    buffer = []

    while words:
        buffer.extend(words[:MAX_LEN])
        words = words[MAX_LEN:]

        if len(buffer) >= MAX_LEN:
            end = find_best_split(buffer, MAX_LEN)
            segments.append({
                "text": ' '.join(buffer[:end]),
                "token_count": len(buffer[:end])
            })
            buffer = buffer[end - OVERLAP:]  # Sovrapposizione

    if buffer:
        segments.append({
            "text": ' '.join(buffer),
            "token_count": len(buffer)
        })

    return segments

# Elaborazione principale - flusso unificato
documents = []
all_segments = []
segment_count = 0

with open(output_segments_path, "w", encoding="utf-8") as segments_file:
    for file_path in document_files:
        # Rilevare l'encoding corretto
        encoding = detect_encoding(file_path)
        if not encoding:
            encoding = "utf-8"  # Fallback su utf-8 se non rilevato

        # Estrai l'ID del documento (plain_XX) e includi anche l'ID della cartella
        file_name = os.path.basename(file_path)
        folder_name = os.path.basename(os.path.dirname(file_path))
        doc_id = f"{folder_name}_{file_name.replace('plain_', '').strip()}"

        buffer = []
        doc_segments = []

        try:
            with open(file_path, "r", encoding=encoding) as f:
                while True:
                    chunk = f.read(CHUNK_SIZE)
                    if not chunk:
                        break

                    # Pulizia e tokenizzazione
                    cleaned_chunk = clean_text(chunk)
                    words = word_tokenize(cleaned_chunk)
                    buffer.extend(words)

                    while len(buffer) >= MAX_LEN:
                        end = find_best_split(buffer, MAX_LEN)
                        segment = buffer[:end]
                        segment_count += 1

                        segment_text = ' '.join(segment)
                        segment_data = {
                            "folder": folder_name,
                            "document": file_name,
                            "segment_number": segment_count,
                            "token_count": len(segment),
                            "text": segment_text,
                            "document_id": doc_id
                        }

                        # Scrivere nel file dei segmenti con formato migliorato
                        segments_file.write(
                            f"Folder: {folder_name} | Document: {file_name} | Segment: {segment_count} | Token Count: {len(segment)}\n{segment_text}\n\n"
                        )

                        # Aggiungere alla lista generale e a quella del documento
                        all_segments.append(segment_data)
                        doc_segments.append(segment_data)

                        buffer = buffer[end - OVERLAP:]

                # Processare l'eventuale buffer rimanente
                if buffer:
                    segment_count += 1
                    segment_text = ' '.join(buffer)
                    segment_data = {
                        "folder": folder_name,
                        "document": file_name,
                        "segment_number": segment_count,
                        "token_count": len(buffer),
                        "text": segment_text,
                        "document_id": doc_id
                    }

                    segments_file.write(
                        f"Folder: {folder_name} | Document: {file_name} | Segment: {segment_count} | Token Count: {len(buffer)}\n{segment_text}\n\n"
                    )

                    all_segments.append(segment_data)
                    doc_segments.append(segment_data)

        except Exception as e:
            print(f"Errore durante la lettura del file {file_path}: {e}")
            continue

        # Preparare i dati del documento
        doc_content = "\n\n".join([segment["text"] for segment in doc_segments])
        documents.append({
            "id": doc_id,
            "content": doc_content,
            "segments": doc_segments
        })

# Salvare tutti i segmenti in un file JSON
with open(output_segments_path.replace(".txt", ".json"), "w", encoding="utf-8") as json_file:
    json.dump(all_segments, json_file, ensure_ascii=False, indent=4)

# Creazione del file di testo unificato
with open(output_txt_path, "w", encoding="utf-8") as txt_file:
    for doc in documents:
        txt_file.write(f"\n--- Document {doc['id']} ---\n")
        txt_file.write(doc['content'] + "\n")

# Salvare la collezione di documenti in JSON
with open(output_json_path, "w", encoding="utf-8") as json_file:
    json.dump(documents, json_file, ensure_ascii=False, indent=4)

print(f"Processo completato! File salvati: {output_txt_path}, {output_json_path}, {output_segments_path}")

Processo completato! File salvati: /content/unified_bandi.txt, /content/unified_bandi.json, /content/testo_segmentato_nltk.txt
